In [1]:
!pip3 install -U peft transformers diffusers accelerate tqdm sentencepiece ipywidgets

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# !pip3 install -U transformers diffusers accelerate tqdm sentencepiece ipywidgets
# !huggingface-cli login
import itertools
import os
import random

import torch
from diffusers import StableDiffusion3Pipeline
from tqdm import tqdm

# Expanded list of cat breeds
cat_breeds = [
    "Abyssinian",
    "Aegean",
    "American Bobtail",
    "American Curl",
    "American Ringtail",
    "American Shorthair",
    "American Wirehair",
    "Aphrodite Giant",
    "Arabian Mau",
    "Asian",
    "Asian Semi-longhair",
    "Australian Mist",
    "Balinese",
    "Bambino",
    "Bengal",
    "Birman",
    "Bombay",
    "Brazilian Shorthair",
    "British Longhair",
    "British Shorthair",
    "Burmese",
    "Burmilla",
    "California Spangled",
    "Chantilly-Tiffany",
    "Chartreux",
    "Chausie",
    "Colorpoint Shorthair",
    "Cornish Rex",
    "Cymric",
    "Cyprus",
    "Devon Rex",
    "Donskoy",
    "Dragon Li",
    "Dwelf",
    "Egyptian Mau",
    "European Shorthair",
    "Exotic Shorthair",
    "Foldex",
    "German Rex",
    "Havana Brown",
    "Highlander",
    "Himalayan",
    "Japanese Bobtail",
    "Javanese",
    "Kanaani",
    "Karelian Bobtail",
    "Khao Manee",
    "Kinkalow",
    "Korat",
    "Korean Bobtail",
    "Korn Ja",
    "Kurilian Bobtail",
    "Lambkin",
    "LaPerm",
    "Lykoi",
    "Maine Coon",
    "Manx",
    "Mekong Bobtail",
    "Minskin",
    "Minuet",
    "Munchkin",
    "Nebelung",
    "Neva Masquerade",
    "Norwegian Forest cat",
    "Ocicat",
    "Ojos Azules",
    "Oriental Bicolor",
    "Oriental Longhair",
    "Oriental Shorthair",
    "Persian",
    "Peterbald",
    "Pixie-bob",
    "Ragamuffin",
    "Ragdoll",
    "Raas",
    "Russian Blue",
    "Russian White, Russian Black and Russian Tabby",
    "Sam Sawet",
    "Savannah",
    "Scottish Fold",
    "Selkirk Rex",
    "Serengeti",
    "Siamese",
    "Siberian",
    "Singapura",
    "Snowshoe",
    "Sokoke",
    "Somali",
    "Sphynx",
    "Suphalak",
    "Thai",
    "Thai Lilac",
    "Tonkinese",
    "Toybob",
    "Toyger",
    "Turkish Angora",
    "Turkish Van",
    "Turkish Vankedisi",
    "Ukrainian Levkoy",
    "York Chocolate",
]

print(f"Cat breeds: {len(cat_breeds)}")

# Expanded list of prepositions
prepositions = ["on", "under", "next to", "beside", "in front of", "behind"]

# Furniture types that fit with the prepositions
furniture_or_outdoor = {
    "on": [
        # Indoor furniture
        "sofa",
        "armchair",
        "dining table",
        "coffee table",
        "bed",
        "bookshelf",
        "cabinet",
        "cupboard",
        "chair",
        "desk",
        "ottoman",
        "recliner",
        "side table",
        "nightstand",
        "TV stand",
        "TV cabinet",
        "end table",
        "love seat",
        "couch",
        "lounge chair",
        "bean bag",
        "bar stool",
        "console table",
        "chest of drawers",
        "vanity",
        "shelf",
        # Outdoor/natural elements
        "rock",
        "tree stump",
        "fence",
        "bench",
        "picnic table",
        "grass",
        "car hood",
        "roof",
        "garden wall",
        "park bench",
        "log",
        "fallen tree",
        "sand dune",
        "boulder",
    ],
    "under": [
        # Indoor furniture
        "sofa",
        "bed",
        "coffee table",
        "side table",
        "desk",
        "recliner",
        "nightstand",
        "chest of drawers",
        "love seat",
        "couch",
        "bookshelf",
        # Outdoor/natural elements
        "tree",
        "bush",
        "bridge",
        "rock",
        "table",
        "park bench",
        "porch",
        "car",
        "wooden deck",
        "fallen tree",
        "shade",
        "overhang",
    ],
    "next to": [
        # Indoor furniture
        "sofa",
        "armchair",
        "dining table",
        "coffee table",
        "bookshelf",
        "cabinet",
        "cupboard",
        "chair",
        "ottoman",
        "recliner",
        "side table",
        "nightstand",
        "TV stand",
        "end table",
        "love seat",
        "lounge chair",
        "console table",
        "vanity",
        "window sill",
        # Outdoor/natural elements
        "tree",
        "bush",
        "fence",
        "bench",
        "log",
        "rock",
        "flower bed",
        "pathway",
        "pond",
        "stream",
        "hill",
        "park bench",
        "fire hydrant",
        "fallen tree",
    ],
    "beside": [
        # Indoor furniture
        "sofa",
        "armchair",
        "dining table",
        "coffee table",
        "bed",
        "bookshelf",
        "chair",
        "ottoman",
        "recliner",
        "side table",
        "nightstand",
        "love seat",
        "couch",
        "lounge chair",
        "vanity",
        # Outdoor/natural elements
        "tree",
        "rock",
        "bush",
        "fence",
        "bench",
        "stream",
        "pathway",
        "log",
        "garden wall",
        "pond",
        "flower bed",
        "hill",
        "fallen tree",
    ],
    "in front of": [
        # Indoor furniture
        "sofa",
        "armchair",
        "coffee table",
        "cabinet",
        "cupboard",
        "TV stand",
        "console table",
        "chest of drawers",
        "bed",
        "recliner",
        "side table",
        "dining chairs",
        "love seat",
        # Outdoor/natural elements
        "tree",
        "rock",
        "fence",
        "bush",
        "bench",
        "stream",
        "pathway",
        "building",
        "garden wall",
        "pond",
        "waterfall",
        "hill",
        "statue",
    ],
    "behind": [
        # Indoor furniture
        "sofa",
        "armchair",
        "bookshelf",
        "recliner",
        "cabinet",
        "TV stand",
        "chest of drawers",
        "love seat",
        # Outdoor/natural elements
        "tree",
        "bush",
        "fence",
        "rock",
        "log",
        "bench",
        "building",
        "hill",
        "statue",
        "garden wall",
        "shed",
    ],
}
camera_angles = [
    "a photo taken from above",
    "a photo taken from below",
    "a side-view photo",
    "a front-facing photo",
    "a photo taken from behind",
]

# List of gaze directions
gaze_directions = [
    "looking straight ahead",
    "looking up",
    "looking down",
    "looking to the left",
    "looking to the right",
    "looking up and to the left",
    "looking up and to the right",
    "looking down and to the left",
    "looking down and to the right",
    "eyes closed",
    "looking over its shoulder",
]
print(f"Prepositions: {len(furniture_or_outdoor)}")
print(
    f"Objects: {len(set(list(itertools.chain.from_iterable(furniture_or_outdoor.values()))))}",
)

2025-02-20 18:42:19.346011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740076939.364181   12330 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740076939.370025   12330 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Cat breeds: 100
Prepositions: 6
Objects: 61


In [3]:
import numpy as np


def set_seeds(seed=42):
    """Fix all possible seeds to ensure reproducibility.
    :param seed: The seed value to be set for all libraries.
    """
    # Python random module
    random.seed(seed)
    # NumPy
    np.random.seed(seed)
    # PyTorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensures deterministic behavior
    torch.backends.cudnn.benchmark = False  # Disables optimizations for reproducibility
    # Environment variable for other libraries or hash-based operations
    os.environ["PYTHONHASHSEED"] = str(seed)


# Example usage
set_seeds(42)

In [7]:
def make_img(folder: str = "./tmp", num_inference_steps=4, guidance_scale=0.0):
    # Directory to save generated images
    output_dir = folder
    os.makedirs(output_dir, exist_ok=True)
    set_seeds(123)
    # Generate synthetic images
    num_images = 5
    for i in tqdm(range(num_images), desc="Generating Images"):
        # Randomly select components for the prompt
        breed = random.choice(cat_breeds)
        preposition = random.choice(prepositions)
        furniture = random.choice(furniture_or_outdoor[preposition])
        angle = random.choice(camera_angles)
        gaze = random.choice(gaze_directions)
        if i == 0:
            angle = "a photo taken from behind"

        # Construct the prompt
        prompt = (
            f"{angle} of a {breed} cat {preposition} the {furniture}, {gaze}. "
            "The cat has realistic fur textures, intricate details, and sharp features, "
            "with soft lighting and a clear focus. The image has a shallow depth of field, "
            "emphasizing the cat in fine detail. 8k, cinematic, photorealistic"
        )
        print(prompt)
        pipe.to(torch.bfloat16)

        try:
            # Generate image
            result = pipe(
                prompt,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                width=512,
                height=512,
            )
            image = result.images[0]  # Get the first image from the list

            # Save the generated image
            output_path = os.path.join(
                output_dir,
                f"small_{i:05d}_cat_{breed}_{preposition}_{furniture}.png",
            )
            image.save(output_path)

        except Exception as e:
            print(f"Failed to generate image {i}: {e}")

    print(f"Generated {num_images} images and saved them in {output_dir}.")

In [8]:
pipe = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-large-turbo",
    torch_dtype=torch.bfloat16,
)
pipe = pipe.to("cuda")
make_img("stable-diffusion-3.5-large-turbo", 4, 0)

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Images:   0%|          | 0/5 [00:00<?, ?it/s]

a photo taken from behind of a American Wirehair cat next to the cabinet, looking to the right. The cat has realistic fur textures, intricate details, and sharp features, with soft lighting and a clear focus. The image has a shallow depth of field, emphasizing the cat in fine detail. 8k, cinematic, photorealistic


  0%|          | 0/4 [00:00<?, ?it/s]

...

Generating Images:  80%|████████  | 4/5 [00:02<00:00,  1.48it/s]

a photo taken from above of a Abyssinian cat beside the pond, eyes closed. The cat has realistic fur textures, intricate details, and sharp features, with soft lighting and a clear focus. The image has a shallow depth of field, emphasizing the cat in fine detail. 8k, cinematic, photorealistic


  0%|          | 0/4 [00:00<?, ?it/s]

Generating Images: 100%|██████████| 5/5 [00:03<00:00,  1.41it/s]

Generated 5 images and saved them in stable-diffusion-3.5-large-turbo.


In [ ]:
# from diffusers import FluxPipeline

# pipe = FluxPipeline.from_pretrained(
#     "black-forest-labs/FLUX.1-schnell",
#     torch_dtype=torch.bfloat16,
# )
# pipe = pipe.to("cuda")
# make_img("FLUX.1-schnell")

In [ ]:
from diffusers import FluxPipeline

pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev",
    torch_dtype=torch.bfloat16,
)
pipe = pipe.to("cuda")
make_img("FLUX.1-dev", 50, 3.5)

In [ ]:
# from diffusers import StableDiffusion3Pipeline

# pipe = StableDiffusion3Pipeline.from_pretrained(
#     "stabilityai/stable-diffusion-3.5-large",
#     torch_dtype=torch.bfloat16,
# )
# pipe = pipe.to("cuda")
# make_img("stable-diffusion-3.5-large", 28, 3.5)

In [ ]:
# import torch
# from diffusers import StableDiffusion3Pipeline

# pipe = StableDiffusion3Pipeline.from_pretrained(
#     "stabilityai/stable-diffusion-3.5-medium",
#     torch_dtype=torch.bfloat16,
# )
# pipe = pipe.to("cuda")

# make_img("stable-diffusion-3.5-medium", 40, 4.5)

In [ ]:
# import torch
# from diffusers import StableDiffusion3Pipeline

# pipe = StableDiffusion3Pipeline.from_pretrained(
#     "stabilityai/stable-diffusion-3-medium-diffusers",
#     torch_dtype=torch.float16,
# )
# pipe = pipe.to("cuda")

# make_img("stable-diffusion-3-medium-diffusers", 28, 7)

In [ ]:
# pipe = DiffusionPipeline.from_pretrained(
#     "stabilityai/stable-diffusion-xl-base-1.0",
#     torch_dtype=torch.float16,
#     use_safetensors=True,
#     variant="fp16",
# )
# pipe.to("cuda")
# make_img("stable-diffusion-xl-base-1.0", 28, 7)

In [ ]:
# import torch
# from diffusers import AutoPipelineForText2Image

# pipe = AutoPipelineForText2Image.from_pretrained(
#     "stabilityai/sdxl-turbo",
#     torch_dtype=torch.float16,
#     variant="fp16",
# )
# pipe.to("cuda")

# make_img("sdxl-turbo", 1, 0)

In [ ]:
# from diffusers import DiffusionPipeline
# import torch

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model_repo_id = "stabilityai/stable-diffusion-3.5-large-turbo"

# torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

# pipe = DiffusionPipeline.from_pretrained(model_repo_id, torch_dtype=torch_dtype)
# pipe = pipe.to(device)

# pipe.load_lora_weights("prithivMLmods/SD3.5-Large-Turbo-HyperRealistic-LoRA", weight_name="SD3.5-4Step-Large-Turbo-HyperRealistic-LoRA.safetensors")
# trigger_word = "hyper realistic"  # Specify trigger word for LoRA
# pipe.fuse_lora(lora_scale=1.0)


In [ ]:
# make_img("real", 1, 0)